In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence

2023-07-12 08:22:33.945335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 08:22:34.104086: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-12 08:22:34.104102: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-12 08:22:34.812529: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
np.array([0,1])

array([0, 1])

# Read passwords from file
Initially we only read the first 100 passwords

If you want to read all the lines, use .readlines()

In [3]:
filename = "darkweb2017-top10000.txt"
nr_lines = 500

In [4]:
passwords = []
comparison_pw = []

with open(filename, "r") as file:
    for i in range(nr_lines):
        passwords.append(next(file).strip())
        comparison_pw.append(next(file).strip())
        

In [5]:
print(passwords)

['123456', '111111', 'qwerty', '12345678', '1234567', '1234567890', '12345', '1q2w3e4r5t', '123456a', 'monkey', 'dragon', '666666', 'myspace1', '121212', '123qwe', 'tinkle', 'gwerty', 'gwerty123', '7777777', '1q2w3e4r', '222222', 'qwerty123', '555555', 'fuckyou', '12345a', '1q2w3e', 'computer', '159753', 'fuckyou1', '789456123', '123654', '888888', 'michael', 'football', '777777', '999999', 'monkey1', 'daniel', 'a12345', '789456', 'love123', 'jordan23', '5201314', 'FQRG7CS493', 'asdf', 'superman', 'love', 'shadow', '333333', 'babygirl1', 'qwert', 'baseball', '0', 'soccer', '131313', '1111111', '0123456789', 'iloveyou2', 'jordan', 'bitch1', 'q1w2e3r4', 'qwer1234', 'soccer1', '101010', 'thomas', 'fuckyou2', 'nicole', '1', 'qazwsxedc', 'andrew', 'apple', 'anthony1', 'money1', 'abc', 'anthony', 'jennifer', 'naruto', '696969', 'joshua', '29rsavoy', 'andrea', 'qwerty12', 'passw0rd', 'hunter', 'welcome', 'superman1', 'xbox360', 'ashley1', 'babygirl', 'trustno1', 'asdf1234', 'buster', 'tigger'

In [6]:
text = "Previous research has shown that language and culture influence what passwords look like. Li et al. (2014) compared Chinese and English passwords, and found several differences. First of all, Chinese speakers prefer to use digits while English speakers prefer letters, especially lowercase letters; around 50% of the Chinese passwords were digits only. Besides that, all users use patterns that they are familiar with. In the case of Chinese speakers, this is pinyin words, while in the case of English speakers, these are English words. Finally, they also find that users tend to use date formats that are customary in their culture (e.g. 0104 or 0401 for April 1st). Wang et al. (2019), who also compared Chinese and English passwords, add to this that there are difference in the letter distributions of password groups. For example, Chinese passwords use the letter q more often than English passwords do, likely because q is a common letter in pinyin, but quite rare in English. These distributions do not exactly match the distributions of the languages completely. For example, they found that the letters l and w occurred more often in passwords than in general Chinese pinyin. They argue that this is likely because they are part of the very popular Chinese names Li and Wang. Furthermore, Wang et al. (2019) add that around 11% of Chinese passwords contain a pinyin name (compared to 4% in their English passwords dataset) and about 31% include a 4+ digit date in their password, of which 17% include a 6 digit date. Note that these percentages are likely overestimations as it is not always possible to determine of a number sequence whether it is a date or refers to something else. All these numbers are significantly higher than in the English password set. Phone numbers are also possible passwords. De Tweede Kamerleden waren het er snel over eens: als ze onderzoek wilden doen naar beïnvloeding van moskeeën, dan móésten ze Lorenzo Vidino wel uitnodigen. De gelauwerde extremisme-deskundige uit de Verenigde Staten had er boeken over geschreven, parlementen voorgelicht, regeringen geadviseerd. Dus leek het een logische keuze voor de Kamercommissie die in 2019 onderzoek deed naar buitenlandse moskeefinanciering, om Vidino te horen als expert. Het werd een „heel interessant gesprek”, herinnert een van de commissieleden zich, die anoniem wil blijven omdat de zitting achter gesloten deuren plaatsvond. „Hij schetste een gedetailleerd beeld van de Nederlandse vertakkingen van de Moslimbroederschap.” De islamitische beweging zou hier volgens Vidino „verdeeldheid” zaaien. Hij kon meerdere Nederlandse moskeeën bij naam noemen die onder invloed zouden staan van de Moslimbroeders en wist zelfs de exacte hoogte van ontvangen giften uit Qatar."


In [7]:
### !!!!!!!!!!!!!!!!!!!!! This is not ideal, this decapitalizes words, so preferably use something else for preprocessing

words = text_to_word_sequence(text, lower=False)

In [8]:
print(len(words))
print(words)

440
['Previous', 'research', 'has', 'shown', 'that', 'language', 'and', 'culture', 'influence', 'what', 'passwords', 'look', 'like', 'Li', 'et', 'al', '2014', 'compared', 'Chinese', 'and', 'English', 'passwords', 'and', 'found', 'several', 'differences', 'First', 'of', 'all', 'Chinese', 'speakers', 'prefer', 'to', 'use', 'digits', 'while', 'English', 'speakers', 'prefer', 'letters', 'especially', 'lowercase', 'letters', 'around', '50', 'of', 'the', 'Chinese', 'passwords', 'were', 'digits', 'only', 'Besides', 'that', 'all', 'users', 'use', 'patterns', 'that', 'they', 'are', 'familiar', 'with', 'In', 'the', 'case', 'of', 'Chinese', 'speakers', 'this', 'is', 'pinyin', 'words', 'while', 'in', 'the', 'case', 'of', 'English', 'speakers', 'these', 'are', 'English', 'words', 'Finally', 'they', 'also', 'find', 'that', 'users', 'tend', 'to', 'use', 'date', 'formats', 'that', 'are', 'customary', 'in', 'their', 'culture', 'e', 'g', '0104', 'or', '0401', 'for', 'April', '1st', 'Wang', 'et', 'al', '

# Feature selection
Build features

- base_features: simple feature set such as the length of the word and amount of characters from different character sets
- levenshtein: calculate the Levensthein distance to most common passwords
- ngrams: ngram of characters as features
- ngram_range: if ngrams is true, this setting determines which ngrams are to be taken into account

In [ ]:
base_features = True
levenshtein = True
ngrams = True
ngram_range = (1, 2)

In [15]:
labels = np.concatenate((np.ones(len(features)), np.zeros(len(features_word))), axis=0)

Passwords are 1 and nonpassword strings are 0

## Levenshtein
To speed up this calculation, it might be worth it to create an implementation of Levenshtein that stops as soon as a distance larger than the current lowest is found. This increases the average time complexity (but not worst case)

In [9]:
from Levenshtein import distance as lev

In [10]:
lev("hoi", "hallo")

4

In [11]:
def calculate_levenshtein_distance(word, passwords):
    low = 42000
    for pw in passwords:
        d = lev(word, pw)
        if d < low:
            low = d
    return low

## Other features

In [12]:

def count_non_repeating(text):
    '''
    Remove repeating letters from a string
    E.g. aaabbbccccccaaa becomes abca
    
    text: input text
    return: text without repeating letters
    '''
    count = 0
    for i, c in enumerate(text):
        if i ==0 or c != text[i-1]:
            count += 1
    return count



def counts(word):
    alpha_lower = 0
    alpha_upper = 0
    numeric = 0
    special = 0
    s = ""
    for c in word:
        if c.islower():
            alpha_lower += 1
            s += 'L'
        elif c.isupper():
            alpha_upper += 1
            s += 'U'
        elif c.isnumeric():
            numeric += 1
            s += 'N'
        else:
            special += 1
            s += 'S'
    length = len(word)
    char_sets = bool(alpha_lower) + bool(alpha_upper) + bool(numeric) + bool(special)
    lev_d = calculate_levenshtein_distance(word, comparison_pw)
    return [length, alpha_lower, alpha_lower/length, alpha_upper, alpha_upper/length, numeric, numeric/length, special, special/length, char_sets, count_non_repeating(s), lev_d]



In [13]:
features = [counts(pw) for pw in passwords]
#print(features)

In [14]:
features_word = [counts(word) for word in words]
#print(features_word)

In [16]:
features = np.concatenate((features, features_word), axis=0)

In [17]:
print(features)

[[6.  0.  0.  ... 1.  1.  1. ]
 [6.  0.  0.  ... 1.  1.  1. ]
 [6.  6.  1.  ... 1.  1.  1. ]
 ...
 [6.  6.  1.  ... 1.  1.  2. ]
 [3.  3.  1.  ... 1.  1.  3. ]
 [5.  4.  0.8 ... 2.  2.  3. ]]


## ngram features

In [18]:
def word2ngrams(word, n=2):
    return [' ' + word[:n-1]] + ["".join(j) for j in zip(*[word[i:] for i in range(n)])] + [word[-(n-1):] + ' ']

In [19]:
print(word2ngrams("test", 2))

ngrams = [word2ngrams(word, n=2) for word in words]

[' t', 'te', 'es', 'st', 't ']


In [20]:
from sklearn.feature_extraction.text import CountVectorizer


In [21]:
vectorizer = CountVectorizer(analyzer = 'char', lowercase = False, ngram_range = ngram_range)

all_words = passwords + words

ngram_features = vectorizer.fit_transform(all_words)
#print(ngram_features)



  (0, 17)	1
  (0, 35)	1
  (0, 48)	1
  (0, 61)	1
  (0, 75)	1
  (0, 90)	1
  (0, 20)	1
  (0, 39)	1
  (0, 52)	1
  (0, 67)	1
  (0, 82)	1
  (1, 17)	6
  (1, 19)	5
  (2, 533)	1
  (2, 639)	1
  (2, 311)	1
  (2, 541)	1
  (2, 589)	1
  (2, 666)	1
  (2, 540)	1
  (2, 646)	1
  (2, 331)	1
  (2, 562)	1
  (2, 609)	1
  (3, 17)	1
  :	:
  (936, 607)	1
  (937, 311)	1
  (937, 589)	1
  (937, 473)	1
  (937, 358)	1
  (937, 388)	1
  (937, 341)	1
  (937, 595)	1
  (937, 365)	1
  (937, 327)	1
  (937, 395)	1
  (937, 355)	1
  (938, 589)	1
  (938, 388)	1
  (938, 610)	1
  (938, 406)	1
  (938, 619)	1
  (939, 541)	1
  (939, 589)	1
  (939, 218)	2
  (939, 191)	1
  (939, 237)	1
  (939, 239)	1
  (939, 592)	1
  (939, 193)	1


In [22]:
print(features.shape)
print(type(features))
print(ngram_features.data.shape)
print(type(ngram_features.data))
# CountVectorizer returns a sparse matrix. This needs to be converted into a dense matrix in order to be able to concatenate it.
features = np.concatenate((np.array(features), ngram_features.toarray()), axis=1)

(940, 12)
<class 'numpy.ndarray'>
(9584,)
<class 'numpy.ndarray'>


In [23]:
# link features and words
total = list(zip(all_words, features))
#print(total)

# Machine Learning shizzle

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn import tree

In [25]:
X_train_tot, X_test_tot, y_train, y_test = train_test_split(total, labels, test_size=0.2, random_state = 42)
X_train = [features for _, features in X_train_tot]
words_train = [words for words, _ in X_train_tot]

X_test = [features for _, features in X_test_tot]
words_test = [words for words, _ in X_test_tot]

print(words_test)

['0401', '101010', 'justin1', 'al', 'marie1', 'of', 'YAgjecc826', 'mickey1', 'stephanie1', 'is', 'green1', 'fuckyou', 'matrix', 'p@ssw0rd', 'onderzoek', 'that', '123654', 'to', 'soccer10', 'qazwsx123', 'in', 'slipknot', '789456', 'love1234', '123456b', 'abcdefg123', 'xbox360', 'family1', 'apple', 'oliver', 'skater1', 'richard', 'Chinese', 'lakers24', 'maverick', 'regeringen', 'or', 'in', 'naruto', 'santiago', 'tend', 'onderzoek', 'mother1', 'dexter', 'that', '$HEX', 'het', 'while', 'gesprek”', 'of', 'which', 'their', 'a', 'of', 'jessie', 'staan', 'mylove', 'george', 'scooter1', 'aaaa', 'that', 'passwords', 'music1', '1', 'soleil', 'blijven', 'destiny', 'poohbear', 'austin1', 'languages', 'zxcvbnm123', 'this', 'orange1', 'lebron23', 'English', 'Chinese', 'do', 'pussy1', 'Li', 'google1', 'naam', 'money1', 'van', 'asdf', '090909', 'find', 'loser1', 'sakura', 'bailey1', 'like', 'than', 'booboo1', 'molly1', '0104', 'booboo', 'mnbvcxz', 'Chinese', 'phoenix', 'sequence', 'wil', 'fucker1', '01

In [26]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [27]:
y_pred = clf.predict(X_test)
print(accuracy_score(y_pred, y_test))

0.8776595744680851


In [28]:
print(confusion_matrix(y_test, y_pred))

[[73  7]
 [16 92]]


In [29]:
for i in range(len(y_test)):
    if y_test[i] != y_pred[i]:
        print(words_test[i])
        print(y_test[i])
        print('')

dexter
1.0

$HEX
1.0

george
1.0

aaaa
1.0

1
1.0

blijven
0.0

poohbear
1.0

asdf
1.0

phoenix
1.0

sequence
0.0

qwer
1.0

Staten
0.0

qweqwe
1.0

deuren
0.0

bij
0.0

vincent
1.0

joshua
1.0

anoniem
0.0

Password
1.0

spongebob
1.0

corvette
1.0

zk.:
1.0

hoogte
0.0

